In [4]:
import torch
from datetime import datetime
import pandas as pd

data = torch.load('processed_data.pt')
timestamps = data['user', 'item'].time.cpu().numpy()
dates = pd.to_datetime(timestamps, unit='s').date
selected_day = datetime.strptime('2017-11-28', '%Y-%m-%d').date()

# Mask for selected day
mask = (dates == selected_day)
behavior = data['user', 'item'].behavior[mask]
num_buy = (behavior == 2).sum().item()
print(f"Buy interactions on {selected_day}: {num_buy}")


Buy interactions on 2017-11-28: 10735


In [5]:
import torch
from torch_geometric.data import HeteroData
import numpy as np

# === STEP 1: Load the full graph ===
data = torch.load('processed_data.pt')

# === STEP 2: Convert timestamps to date and filter for one day ===
# Assume time is stored in seconds in edge attribute 'time'
timestamps = data['user', 'item'].time.cpu().numpy()
dates = pd.to_datetime(timestamps, unit='s').date
selected_day = datetime.strptime('2017-11-28', '%Y-%m-%d').date()

# Create mask where edge date matches selected day
mask = np.array(dates) == selected_day
mask = torch.from_numpy(mask).to(torch.bool)

# === STEP 3: Filter edges for that day ===
edge_index = data['user', 'item'].edge_index[:, mask]
behavior = data['user', 'item'].behavior[mask]
time = data['user', 'item'].time[mask]

# === STEP 4: Create a new HeteroData with only those edges ===
filtered_data = HeteroData()
filtered_data['user'].num_nodes = data['user'].num_nodes
filtered_data['item'].num_nodes = data['item'].num_nodes
filtered_data['category'].num_nodes = data['category'].num_nodes

filtered_data['user', 'item'].edge_index = edge_index
filtered_data['user', 'item'].behavior = behavior
filtered_data['user', 'item'].time = time

# Copy item-category edges as-is (optional)
filtered_data['item', 'category'].edge_index = data['item', 'category'].edge_index

# === STEP 5: Save or use the filtered graph ===
print(f"Filtered graph for {selected_day} has {edge_index.shape[1]} edges.")
torch.save(filtered_data, f'graph_{selected_day}.pt')


Filtered graph for 2017-11-28 has 490128 edges.
